## LionAGI - 105

Interacting with external environment. 

AI models such as LLM, have a "knowledge cutoff" meaning they do not have access to information after they have been trained. This is a problem, 

especially because neural network inheritantly have **"hallucination"** problem.

How can we solve this?

One way is to feed LLM with the best information so that it can give more appropriate response due to the additional information, context. This approach is called **Retrieval Augmented Generation (RAG)**.

In this tutorial we will walk through how to use llamaindex as a retrieval tool to let a LLM interact with github repositories.

----

In [1]:
from IPython.display import display, Markdown


def printmd(string):
    display(Markdown(string))


from pathlib import Path

datapath = Path.cwd() / "data" / "lion-core-source"

### Install LLamaIndex (RAG Tool provider)

In [2]:
# %pip install lionagi llama-index nbconvert

In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")

docs = SimpleDirectoryReader(
    input_dir=datapath, recursive=True, required_exts=[".py", ".md", ".ipynb"]
).load_data()

index = VectorStoreIndex.from_documents(documents=docs)
query_engine = index.as_query_engine(llm=llm)

In [4]:
response = query_engine.query(
    "What is the difference between an lion-core Note and a dictionary??",
)


printmd(response.response)

A `Note` in the Lion framework is a flexible container specifically designed for managing nested dictionary data structures, allowing for complex and hierarchical information storage. In contrast, a dictionary is a standard data structure in Python that stores key-value pairs without the additional methods and functionalities provided by the `Note` class for manipulating and accessing nested data. The `Note` class enhances the capabilities of a dictionary by offering a rich set of methods tailored for hierarchical data management.

### Create Tool for Branch

great. 

let's add this tool to our branch such that, if needed, it can refer to `lion-core` repo for future conversations. 

We define a tool in LION system by wrapping it as a python function with docstring as tool manual.

In [5]:
from lionfuncs import to_dict


async def query_lion_core(query: str):
    """
    Use this tool to query the lion-core repository for information.
    you should frequently refer to this if you are unsure about anything.

    Args:
        query: str: A question, term or query about lion-core.
    """
    response = await query_engine.aquery(query)
    return to_dict(
        response,
        recursive=True,
        recursive_python_only=False,
        max_recursive_depth=4,
    )

### Let us try it out

In [6]:
question1 = "Articulate the various features and mechanism behind branch in lion-core? You must use query tool to help."

In [7]:
from lionagi import Branch

branch = Branch(
    system="you are a QA bot with access to real code",
    tools=query_lion_core,
)

await branch.chat(instruction=question1, tools=True)

we can check all the messages directly related to branch by the following ways

In [8]:
# convert messages directly into a pandas dataframe
# df = branch.messages.to_df()

In [9]:
# convert formatteded messages into a pandas dataframe
df = branch.to_df()
df

,ln_id,message_type,timestamp,role,content,metadata,sender,recipient
0,0536326e2ed90e795dc300b47bc4e557,System,2024-10-08T16:41:49.304167,system,{'system_info': 'you are a QA bot with access ...,{'last_updated': {'recipient': '2024-10-08T16:...,system,46ce6f32efc98c1b3b27da6c8a019a28
1,d9076cde3f34ead74d9b62397000e25d,Instruction,2024-10-08T16:41:49.304765,user,{'instruction': 'Articulate the various featur...,{'last_updated': {'sender': '2024-10-08T16:41:...,user,46ce6f32efc98c1b3b27da6c8a019a28
2,0733df0c91bdea10652092c85bbaed7f,AssistantResponse,2024-10-08T16:41:49.997432,assistant,{'assistant_response': None},{'last_updated': {'sender': '2024-10-08T16:41:...,46ce6f32efc98c1b3b27da6c8a019a28,user
3,7d31a9616741039a161c06affe6f6094,ActionRequest,2024-10-08T16:41:50.000540,assistant,{'action_request': {'function': 'query_lion_co...,{'last_updated': {'function': '2024-10-08T16:4...,46ce6f32efc98c1b3b27da6c8a019a28,8840df96fdaa62fcef36c72625af90bf
4,601c46b2bf263f0a90bf682aaf49b05f,ActionResponse,2024-10-08T16:41:51.943388,assistant,{'action_response': {'function': 'query_lion_c...,{'last_updated': {'function': '2024-10-08T16:4...,8840df96fdaa62fcef36c72625af90bf,46ce6f32efc98c1b3b27da6c8a019a28


In [10]:
# now let's examine the various messages assisatnt sent back

from lionfuncs import as_readable_json

for i in branch.messages:
    if i.role == "assistant":
        content = to_dict(
            i.content,
            recursive=True,
            recursive_python_only=False,
            max_recursive_depth=5,
        )
        printmd(as_readable_json(content) + "\n\n --- \n")

{
    "assistant_response": {}
}

 --- 


{
    "action_request": {
        "function": "query_lion_core",
        "arguments": {
            "query": "branch features and mechanism in lion-core"
        }
    }
}

 --- 


{
    "action_response": {
        "function": "query_lion_core",
        "arguments": {
            "query": "branch features and mechanism in lion-core"
        },
        "output": {
            "response": "The `Branch` class in the Lion framework is designed to represent a branch in the conversation tree. It manages messages, tools, and communication within that branch. This class offers a comprehensive set of methods that facilitate the handling of various types of interactions and data, enhancing the overall functionality and user experience within the framework.",
            "source_nodes": [
                {
                    "node": {
                        "id_": "5cd1692b-4860-4724-b04c-4edb977f3abb",
                        "embedding": null,
                        "metadata": {
                            "file_path": "/Users/lion/playground/tutorials/data/lion-core-source/md_docs/api_reference/branch.md",
                            "file_name": "branch.md",
                            "file_type": "text/markdown",
                            "file_size": 5919,
                            "creation_date": "2024-10-08",
                            "last_modified_date": "2024-10-08"
                        },
                        "excluded_embed_metadata_keys": [
                            "file_name",
                            "file_type",
                            "file_size",
                            "creation_date",
                            "last_modified_date",
                            "last_accessed_date"
                        ],
                        "excluded_llm_metadata_keys": [
                            "file_name",
                            "file_type",
                            "file_size",
                            "creation_date",
                            "last_modified_date",
                            "last_accessed_date"
                        ],
                        "relationships": {
                            "1": {
                                "node_id": "69cbd6d9-a78a-44f9-8ba7-ae5826dbb834",
                                "node_type": "4",
                                "metadata": {
                                    "file_path": "/Users/lion/playground/tutorials/data/lion-core-source/md_docs/api_reference/branch.md",
                                    "file_name": "branch.md",
                                    "file_type": "text/markdown",
                                    "file_size": 5919,
                                    "creation_date": "2024-10-08",
                                    "last_modified_date": "2024-10-08"
                                },
                                "hash": "0e884b953536e98b170a81a7318b756d382e48641c6c74876fccde767173dea3",
                                "class_name": "RelatedNodeInfo"
                            }
                        },
                        "text": "Overview\n\nThe `Branch` class represents a branch in the conversation tree within the Lion framework. It manages messages, tools, and communication within the branch, providing a rich set of methods for handling various types of interactions and data.",
                        "mimetype": "text/plain",
                        "start_char_idx": 2,
                        "end_char_idx": 252,
                        "text_template": "{metadata_str}\n\n{content}",
                        "metadata_template": "{key}: {value}",
                        "metadata_seperator": "\n",
                        "class_name": "TextNode"
                    },
                    "score": 0.8207475210721997,
                    "class_name": "NodeWithScore"
                },
                {
                    "node": {
                        "id_": "fef1c5c5-d14b-45dc-8b2c-92b2588716ea",
                        "embedding": null,
                        "metadata": {
                            "file_path": "/Users/lion/playground/tutorials/data/lion-core-source/README.md",
                            "file_name": "README.md",
                            "file_type": "text/markdown",
                            "file_size": 8855,
                            "creation_date": "2024-10-08",
                            "last_modified_date": "2024-10-08"
                        },
                        "excluded_embed_metadata_keys": [
                            "file_name",
                            "file_type",
                            "file_size",
                            "creation_date",
                            "last_modified_date",
                            "last_accessed_date"
                        ],
                        "excluded_llm_metadata_keys": [
                            "file_name",
                            "file_type",
                            "file_size",
                            "creation_date",
                            "last_modified_date",
                            "last_accessed_date"
                        ],
                        "relationships": {
                            "1": {
                                "node_id": "5766a4d6-102d-43c3-a297-cc1e3f923e3e",
                                "node_type": "4",
                                "metadata": {
                                    "file_path": "/Users/lion/playground/tutorials/data/lion-core-source/README.md",
                                    "file_name": "README.md",
                                    "file_type": "text/markdown",
                                    "file_size": 8855,
                                    "creation_date": "2024-10-08",
                                    "last_modified_date": "2024-10-08"
                                },
                                "hash": "8bbc88734958ad2c4897ac6b0494164ffbeb1607d13ab36727716ddbcc4d276e",
                                "class_name": "RelatedNodeInfo"
                            }
                        },
                        "text": "\ud83d\ude80 Features",
                        "mimetype": "text/plain",
                        "start_char_idx": 2,
                        "end_char_idx": 12,
                        "text_template": "{metadata_str}\n\n{content}",
                        "metadata_template": "{key}: {value}",
                        "metadata_seperator": "\n",
                        "class_name": "TextNode"
                    },
                    "score": 0.8169916159758592,
                    "class_name": "NodeWithScore"
                }
            ],
            "metadata": {
                "5cd1692b-4860-4724-b04c-4edb977f3abb": {
                    "file_path": "/Users/lion/playground/tutorials/data/lion-core-source/md_docs/api_reference/branch.md",
                    "file_name": "branch.md",
                    "file_type": "text/markdown",
                    "file_size": 5919,
                    "creation_date": "2024-10-08",
                    "last_modified_date": "2024-10-08"
                },
                "fef1c5c5-d14b-45dc-8b2c-92b2588716ea": {
                    "file_path": "/Users/lion/playground/tutorials/data/lion-core-source/README.md",
                    "file_name": "README.md",
                    "file_type": "text/markdown",
                    "file_size": 8855,
                    "creation_date": "2024-10-08",
                    "last_modified_date": "2024-10-08"
                }
            }
        }
    }
}

 --- 


In [11]:
branch.messages[-1].content["action_response"]["output"]

{'response': 'The `Branch` class in the Lion framework is designed to represent a branch in the conversation tree. It manages messages, tools, and communication within that branch. This class offers a comprehensive set of methods that facilitate the handling of various types of interactions and data, enhancing the overall functionality and user experience within the framework.',
 'source_nodes': [{'node': {'id_': '5cd1692b-4860-4724-b04c-4edb977f3abb',
    'embedding': None,
    'metadata': {'file_path': '/Users/lion/playground/tutorials/data/lion-core-source/md_docs/api_reference/branch.md',
     'file_name': 'branch.md',
     'file_type': 'text/markdown',
     'file_size': 5919,
     'creation_date': '2024-10-08',
     'last_modified_date': '2024-10-08'},
    'excluded_embed_metadata_keys': ['file_name',
     'file_type',
     'file_size',
     'creation_date',
     'last_modified_date',
     'last_accessed_date'],
    'excluded_llm_metadata_keys': ['file_name',
     'file_type',
   

### finally you can save all these messages

In [12]:
from pathlib import Path
from lionfuncs import create_path

fp = create_path(
    directory=Path(".") / "data",
    filename="lion_core_branch_features",
    extension=".csv",
    random_hash_digits=5,  # we add random hash digits to ensure file uniqueness
)

df = branch.to_df()
df.to_csv(fp, index=False)
print(f"Data saved to {fp}")

Data saved to data/lion_core_branch_features-c98da.csv
